# 要約 
このJupyterノートブックでは、4ビット量子化された「Gemma-2 9b Instruct」モデルに基づくLoRAアダプターを使用した推論の手法が示されています。主な目的は、Chatbot Arenaコンペティションにおける、2つの言語モデル間のユーザー応答の選好予測を行うことです。

### 問題の扱い
ノートブックは、LoRAアダプターを用いて、量子化による誤差の影響を軽減しつつ、推論を迅速化する方法を探求しています。モデルをマージすることで誤差が生じる可能性があるため、LoRAアダプターを維持した状態での推論を推奨しています。また、モデルの性能は評価セットでの対数損失が0.9371、公開リーダーボードでの対数損失が0.941であると報告しています。

### 使用ライブラリと手法
ノートブックでは、主に以下のライブラリが使用されています:
- **Transformers**: 「Gemma2ForSequenceClassification」と「GemmaTokenizerFast」を使用してモデルの呼び出しやトークン化を実施。
- **Peft**: LoRAアダプターの適用に用いられます。
- **Torch**: GPU利用や自動混合精度計算をサポートし、モデルの推論が行われます。
- **PandasとNumPy**: データ処理と操作のためにデータフレームを使用し、結果の格納に役立てています。

### データ処理と推論
データはCSVファイルから読み込まれ、テキストの前処理を行った後、トークナイズされます。推論は、2つのGPUを用いてバッチごとに行われ、モデルAとモデルBの各々の勝率を計算します。結果として、モデルごとの勝率や引き分け確率をデータフレームに格納し、最終的に提出用のCSVファイルとして成果物を保存します。

全体として、このノートブックは、量子化されたモデルを最適に利用するための手法と細かな設定を提供し、効率的に予測を行うフレームワークを構築しています。

---


# 用語概説 
以下は、Jupyter Notebook内で使用されている専門用語の簡単な解説です。これらの用語は、初心者が特に理解しにくい可能性があるものです。

1. **Gemma-2**: 大規模言語モデル（LLM）の一種で、テキスト生成や分類タスクに用いられます。特に今回のノートブックでは、「Gemma-2 9b Instruct」というバージョンが使用されています。

2. **LoRA (Low-Rank Adaptation)**: 転移学習やファインチューニングを行うための手法で、もともとのモデルの重みを劣化させることなく、新しい情報を効率的に学習するために低ランクの更新を加える手法です。

3. **量子化 (Quantization)**: モデルの重みやアクティベーションの精度を減らして、メモリ使用量を削減し、計算効率を向上させる技術です。特に4ビット量子化は、重みを4ビットで表現することを意味します。

4. **アテンションマスク (Attention Mask)**: モデルに入力されるデータのどの部分を注目すべきかを示すバイナリのマスクです。通常は、パディングされた部分（無視すべき部分）を除外するために使われます。

5. **トークナイズ (Tokenization)**: 文章やテキストを処理可能な単位（トークン）に分解するプロセスです。これにより、テキストがモデルに入力できる形式に変換されます。

6. **トークンID (Token ID)**: トークナイズされた各トークンに対応する整数値で、単語やサブワードを表現します。

7. **ソフトマックス (Softmax)**: 複数のクラスに対する確率を計算する関数です。出力層で使用され、各クラスのスコアを確率に変換します。

8. **TTA (Test-Time Augmentation)**: テストデータに対してデータ拡張を適用する手法で、モデルの予測の安定性を向上させます。複数の異なる視点やデータの拡張から得られる予測を平均化することが一般的です。

9. **Pad_without_fast_tokenizer_warning**: 特定のトークナイザー使用時に、パディングに関する警告を避けるための関数です。入力のパディングを行う際には、慎重に扱う必要があります。

10. **自動混合精度 (Automatic Mixed Precision)**: 計算の精度を自動的に調整する技術で、処理速度を向上させるために、必要に応じて32ビット（float32）と16ビット（float16）の演算を切り替えます。

11. **キャッシュ (Cache)**: 計算結果を保存しておき、再使用することで計算を効率化すること。特に推論時間を短縮するために有効です。

これらの用語は、特にこのノートブックやその設定において重要な部分を占めており、理解を深める手助けとなります。

---


<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## What this notebook is

This is a inference notebook using 4-bit quantized [Gemma-2 9b Instruct](https://blog.google/technology/developers/google-gemma-2/) and a LoRA adapter trained using the script I uploaded [here](https://www.kaggle.com/code/emiz6413/gemma-2-9b-4-bit-qlora-finetune).
Although we can choose to merge the LoRA adapter to the base model for faster inference, naively doing so could introduce non-negligible quantization error. Therefore, I opted to keep the LoRA adapter unmerged. 

## Result

| subset | log loss |
| - | - |
| eval set | 0.9371 |
| public LB | 0.941 |

The submission takes around 4 hours with `max_length=2048` without TTA.

</div>
<div class="column-right">

# 日本語訳

## このノートブックについて

これは、4ビット量子化された [Gemma-2 9b Instruct](https://blog.google/technology/developers/google-gemma-2/) と、私がアップロードしたスクリプトを使用してトレーニングしたLoRAアダプターを利用した推論ノートブックです [ここ](https://www.kaggle.com/code/emiz6413/gemma-2-9b-4-bit-qlora-finetune)で確認できます。
LoRAアダプターをベースモデルにマージすることで推論を速くすることもできますが、安易にそうすると無視できない量子化誤差が発生する可能性があります。そのため、私はLoRAアダプターをマージせずに維持することにしました。

## 結果

| サブセット | 対数損失 |
| - | - |
| 評価セット | 0.9371 |
| 公開LB | 0.941 |

提出には約4時間かかります。`max_length=2048`でTTAは使用していません。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!pip install transformers peft accelerate bitsandbytes \
    -U --no-index --find-links /kaggle/input/lmsys-wheel-files
```

</div>
<div class="column-right">

# 日本語訳

```python
!pip install transformers peft accelerate bitsandbytes \
    -U --no-index --find-links /kaggle/input/lmsys-wheel-files
```

</div>
</details>

In [ ]:
!pip install transformers peft accelerate bitsandbytes \
    -U --no-index --find-links /kaggle/input/lmsys-wheel-files

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import time
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor

import torch
import sklearn
import numpy as np
import pandas as pd
from transformers import Gemma2ForSequenceClassification, GemmaTokenizerFast, BitsAndBytesConfig
from transformers.data.data_collator import pad_without_fast_tokenizer_warning
from peft import PeftModel
```

</div>
<div class="column-right">

# 日本語訳

```python
import time
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor

import torch
import sklearn
import numpy as np
import pandas as pd
from transformers import Gemma2ForSequenceClassification, GemmaTokenizerFast, BitsAndBytesConfig
from transformers.data.data_collator import pad_without_fast_tokenizer_warning
from peft import PeftModel
```

</div>
</details>

In [ ]:
import time
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor

import torch
import sklearn
import numpy as np
import pandas as pd
from transformers import Gemma2ForSequenceClassification, GemmaTokenizerFast, BitsAndBytesConfig
from transformers.data.data_collator import pad_without_fast_tokenizer_warning
from peft import PeftModel

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
assert torch.cuda.device_count() == 2
```

</div>
<div class="column-right">

# 日本語訳

```python
assert torch.cuda.device_count() == 2
```

</div>
</details>

In [ ]:
assert torch.cuda.device_count() == 2

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Configurations

</div>
<div class="column-right">

# 日本語訳

## 設定


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
@dataclass
class Config:
    gemma_dir = '/kaggle/input/gemma-2/transformers/gemma-2-9b-it-4bit/1/gemma-2-9b-it-4bit'
    lora_dir = '/kaggle/input/73zap2gx/checkpoint-5748'
    max_length = 2048
    batch_size = 4
    device = torch.device("cuda")    
    tta = False  # test time augmentation. <prompt>-<model-b's response>-<model-a's response>
    spread_max_length = False  # whether to apply max_length//3 on each input or max_length on the concatenated input

cfg = Config()
```

</div>
<div class="column-right">

# 日本語訳

```python
@dataclass
class Config:
    gemma_dir = '/kaggle/input/gemma-2/transformers/gemma-2-9b-it-4bit/1/gemma-2-9b-it-4bit'
    lora_dir = '/kaggle/input/73zap2gx/checkpoint-5748'
    max_length = 2048
    batch_size = 4
    device = torch.device("cuda")    
    tta = False  # テスト時のデータ拡張。<prompt>-<model-bの応答>-<model-aの応答>
    spread_max_length = False  # 各入力にmax_length//3を適用するか、連結した入力にmax_lengthを適用するか

cfg = Config()
```

</div>
</details>

In [ ]:
@dataclass
class Config:
    gemma_dir = '/kaggle/input/gemma-2/transformers/gemma-2-9b-it-4bit/1/gemma-2-9b-it-4bit'
    lora_dir = '/kaggle/input/73zap2gx/checkpoint-5748'
    max_length = 2048
    batch_size = 4
    device = torch.device("cuda")    
    tta = False  # テスト時のデータ拡張。<prompt>-<model-bの応答>-<model-aの応答>
    spread_max_length = False  # 各入力にmax_length//3を適用するか、連結した入力にmax_lengthを適用するか

cfg = Config()

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Load & pre-process Data 

</div>
<div class="column-right">

# 日本語訳

# データの読み込みと前処理


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')
```

</div>
<div class="column-right">

# 日本語訳

```python
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')
```

</div>
</details>

In [ ]:
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def process_text(text: str) -> str:
    return " ".join(eval(text, {"null": ""}))

test.loc[:, 'prompt'] = test['prompt'].apply(process_text)
test.loc[:, 'response_a'] = test['response_a'].apply(process_text)
test.loc[:, 'response_b'] = test['response_b'].apply(process_text)

display(test.head(5))
```

</div>
<div class="column-right">

# 日本語訳

```python
def process_text(text: str) -> str:
    return " ".join(eval(text, {"null": ""}))

test.loc[:, 'prompt'] = test['prompt'].apply(process_text)
test.loc[:, 'response_a'] = test['response_a'].apply(process_text)
test.loc[:, 'response_b'] = test['response_b'].apply(process_text)

display(test.head(5))
```

</div>
</details>

In [ ]:
def process_text(text: str) -> str:
    return " ".join(eval(text, {"null": ""}))

test.loc[:, 'prompt'] = test['prompt'].apply(process_text)
test.loc[:, 'response_a'] = test['response_a'].apply(process_text)
test.loc[:, 'response_b'] = test['response_b'].apply(process_text)

display(test.head(5))

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Tokenize

</div>
<div class="column-right">

# 日本語訳

# トークナイズ


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def tokenize(
    tokenizer, prompt, response_a, response_b, max_length=cfg.max_length, spread_max_length=cfg.spread_max_length
):
    prompt = ["<prompt>: " + p for p in prompt]
    response_a = ["\n\n<response_a>: " + r_a for r_a in response_a]
    response_b = ["\n\n<response_b>: " + r_b for r_b in response_b]
    if spread_max_length:
        prompt = tokenizer(prompt, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_a = tokenizer(response_a, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_b = tokenizer(response_b, max_length=max_length//3, truncation=True, padding=False).input_ids
        input_ids = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        attention_mask = [[1]* len(i) for i in input_ids]
    else:
        text = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        tokenized = tokenizer(text, max_length=max_length, truncation=True, padding=False)
        input_ids = tokenized.input_ids
        attention_mask = tokenized.attention_mask
    return input_ids, attention_mask
```

</div>
<div class="column-right">

# 日本語訳

```python
def tokenize(
    tokenizer, prompt, response_a, response_b, max_length=cfg.max_length, spread_max_length=cfg.spread_max_length
):
    prompt = ["<prompt>: " + p for p in prompt]  # プロンプトに"<prompt>: "を追加
    response_a = ["\n\n<response_a>: " + r_a for r_a in response_a]  # 応答Aにプレフィックスを追加
    response_b = ["\n\n<response_b>: " + r_b for r_b in response_b]  # 応答Bにプレフィックスを追加
    if spread_max_length:  # spread_max_lengthがTrueの場合
        # 各要素をmax_length//3でトークナイズ
        prompt = tokenizer(prompt, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_a = tokenizer(response_a, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_b = tokenizer(response_b, max_length=max_length//3, truncation=True, padding=False).input_ids
        input_ids = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]  # 各リストを結合
        attention_mask = [[1]* len(i) for i in input_ids]  # 各入力の長さに応じたアテンションマスクを作成
    else:
        # 各要素を結合してトークン化
        text = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        tokenized = tokenizer(text, max_length=max_length, truncation=True, padding=False)  # トークナイズ
        input_ids = tokenized.input_ids  # トークンIDを取得
        attention_mask = tokenized.attention_mask  # アテンションマスクを取得
    return input_ids, attention_mask  # トークンIDとアテンションマスクを返す
```

</div>
</details>

In [ ]:
def tokenize(
    tokenizer, prompt, response_a, response_b, max_length=cfg.max_length, spread_max_length=cfg.spread_max_length
):
    prompt = ["<prompt>: " + p for p in prompt]  # プロンプトに"<prompt>: "を追加
    response_a = ["\n\n<response_a>: " + r_a for r_a in response_a]  # 応答Aにプレフィックスを追加
    response_b = ["\n\n<response_b>: " + r_b for r_b in response_b]  # 応答Bにプレフィックスを追加
    if spread_max_length:  # spread_max_lengthがTrueの場合
        # 各要素をmax_length//3でトークナイズ
        prompt = tokenizer(prompt, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_a = tokenizer(response_a, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_b = tokenizer(response_b, max_length=max_length//3, truncation=True, padding=False).input_ids
        input_ids = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]  # 各リストを結合
        attention_mask = [[1]* len(i) for i in input_ids]  # 各入力の長さに応じたアテンションマスクを作成
    else:
        # 各要素を結合してトークン化
        text = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        tokenized = tokenizer(text, max_length=max_length, truncation=True, padding=False)  # トークナイズ
        input_ids = tokenized.input_ids  # トークンIDを取得
        attention_mask = tokenized.attention_mask  # アテンションマスクを取得
    return input_ids, attention_mask  # トークンIDとアテンションマスクを返す

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
%%time

tokenizer = GemmaTokenizerFast.from_pretrained(cfg.gemma_dir)
tokenizer.add_eos_token = True
tokenizer.padding_side = "right"

data = pd.DataFrame()
data["id"] = test["id"]
data["input_ids"], data["attention_mask"] = tokenize(tokenizer, test["prompt"], test["response_a"], test["response_b"])
data["length"] = data["input_ids"].apply(len)

aug_data = pd.DataFrame()
aug_data["id"] = test["id"]
# swap response_a & response_b
aug_data['input_ids'], aug_data['attention_mask'] = tokenize(tokenizer, test["prompt"], test["response_b"], test["response_a"])
aug_data["length"] = aug_data["input_ids"].apply(len)
```

</div>
<div class="column-right">

# 日本語訳

```python
%%time

tokenizer = GemmaTokenizerFast.from_pretrained(cfg.gemma_dir)  # Gemmaトークナイザーを読み込む
tokenizer.add_eos_token = True  # 終了トークンを追加
tokenizer.padding_side = "right"  # パディングの位置を右に設定

data = pd.DataFrame()
data["id"] = test["id"]
data["input_ids"], data["attention_mask"] = tokenize(tokenizer, test["prompt"], test["response_a"], test["response_b"])  # トークン化した結果をデータフレームに格納
data["length"] = data["input_ids"].apply(len)  # 各入力の長さを計算して追加

aug_data = pd.DataFrame()  # 拡張データ用のデータフレームを作成
aug_data["id"] = test["id"]
# response_aとresponse_bを入れ替える
aug_data['input_ids'], aug_data['attention_mask'] = tokenize(tokenizer, test["prompt"], test["response_b"], test["response_a"])  # トークナイズして格納
aug_data["length"] = aug_data["input_ids"].apply(len)  # 長さを計算
```

</div>
</details>

In [ ]:
%%time

tokenizer = GemmaTokenizerFast.from_pretrained(cfg.gemma_dir)  # Gemmaトークナイザーを読み込む
tokenizer.add_eos_token = True  # 終了トークンを追加
tokenizer.padding_side = "right"  # パディングの位置を右に設定

data = pd.DataFrame()
data["id"] = test["id"]
data["input_ids"], data["attention_mask"] = tokenize(tokenizer, test["prompt"], test["response_a"], test["response_b"])  # トークン化した結果をデータフレームに格納
data["length"] = data["input_ids"].apply(len)  # 各入力の長さを計算して追加

aug_data = pd.DataFrame()  # 拡張データ用のデータフレームを作成
aug_data["id"] = test["id"]
# response_aとresponse_bを入れ替える
aug_data['input_ids'], aug_data['attention_mask'] = tokenize(tokenizer, test["prompt"], test["response_b"], test["response_a"])  # トークナイズして格納
aug_data["length"] = aug_data["input_ids"].apply(len)  # 長さを計算

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
print(tokenizer.decode(data["input_ids"][0]))
```

</div>
<div class="column-right">

# 日本語訳

```python
print(tokenizer.decode(data["input_ids"][0]))  # トークナイズしたデータの最初の要素をデコードして表示
```

</div>
</details>

In [ ]:
print(tokenizer.decode(data["input_ids"][0]))  # トークナイズしたデータの最初の要素をデコードして表示

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
print(tokenizer.decode(aug_data["input_ids"][0]))
```

</div>
<div class="column-right">

# 日本語訳

```python
print(tokenizer.decode(aug_data["input_ids"][0]))  # 拡張データの最初の要素をデコードして表示
```

</div>
</details>

In [ ]:
print(tokenizer.decode(aug_data["input_ids"][0]))  # 拡張データの最初の要素をデコードして表示

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Load model

</div>
<div class="column-right">

# 日本語訳

# モデルを読み込む


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Load base model on GPU 0
device_0 = torch.device('cuda:0')
model_0 = Gemma2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,
    device_map=device_0,
    use_cache=False,
)

# Load base model on GPU 1
device_1 = torch.device('cuda:1')
model_1 = Gemma2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,
    device_map=device_1,
    use_cache=False,
)
```

</div>
<div class="column-right">

# 日本語訳

```python
# GPU 0にベースモデルを読み込む
device_0 = torch.device('cuda:0')
model_0 = Gemma2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,
    device_map=device_0,
    use_cache=False,
)

# GPU 1にベースモデルを読み込む
device_1 = torch.device('cuda:1')
model_1 = Gemma2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,
    device_map=device_1,
    use_cache=False,
)
```

</div>
</details>

In [ ]:
# GPU 0にベースモデルを読み込む
device_0 = torch.device('cuda:0')
model_0 = Gemma2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,
    device_map=device_0,
    use_cache=False,
)

# GPU 1にベースモデルを読み込む
device_1 = torch.device('cuda:1')
model_1 = Gemma2ForSequenceClassification.from_pretrained(
    cfg.gemma_dir,
    device_map=device_1,
    use_cache=False,
)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

#### Load LoRA adapter

</div>
<div class="column-right">

# 日本語訳

#### LoRAアダプターを読み込む


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
model_0 = PeftModel.from_pretrained(model_0, cfg.lora_dir)
model_1 = PeftModel.from_pretrained(model_1, cfg.lora_dir)
```

</div>
<div class="column-right">

# 日本語訳

```python
model_0 = PeftModel.from_pretrained(model_0, cfg.lora_dir)  # LoRAアダプターをモデル0に適用
model_1 = PeftModel.from_pretrained(model_1, cfg.lora_dir)  # LoRAアダプターをモデル1に適用
```

</div>
</details>

In [ ]:
model_0 = PeftModel.from_pretrained(model_0, cfg.lora_dir)  # LoRAアダプターをモデル0に適用
model_1 = PeftModel.from_pretrained(model_1, cfg.lora_dir)  # LoRAアダプターをモデル1に適用

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Inference


</div>
<div class="column-right">

# 日本語訳

# 推論



</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
@torch.no_grad()
@torch.cuda.amp.autocast()
def inference(df, model, device, batch_size=cfg.batch_size, max_length=cfg.max_length):
    a_win, b_win, tie = [], [], []
    
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))
        tmp = df.iloc[start_idx:end_idx]
        input_ids = tmp["input_ids"].to_list()
        attention_mask = tmp["attention_mask"].to_list()
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding="longest",
            pad_to_multiple_of=None,
            return_tensors="pt",
        )
        outputs = model(**inputs.to(device))
        proba = outputs.logits.softmax(-1).cpu()
        
        a_win.extend(proba[:, 0].tolist())
        b_win.extend(proba[:, 1].tolist())
        tie.extend(proba[:, 2].tolist())
    
    df["winner_model_a"] = a_win
    df["winner_model_b"] = b_win
    df["winner_tie"] = tie
    
    return df
```

</div>
<div class="column-right">

# 日本語訳

```python
@torch.no_grad()  # 勾配計算を無効にする
@torch.cuda.amp.autocast()  # 自動混合精度を使って演算を行う
def inference(df, model, device, batch_size=cfg.batch_size, max_length=cfg.max_length):
    a_win, b_win, tie = [], [], []  # 各モデルの勝率と引き分けを記録するリスト
    
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))  # バッチの終端インデックスを計算
        tmp = df.iloc[start_idx:end_idx]  # データフレームからバッチを取得
        input_ids = tmp["input_ids"].to_list()  # 入力IDを取得
        attention_mask = tmp["attention_mask"].to_list()  # アテンションマスクを取得
        # トークナイザーを使ってデータをパディング
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding="longest",
            pad_to_multiple_of=None,
            return_tensors="pt",
        )
        outputs = model(**inputs.to(device))  # モデルで出力を計算
        proba = outputs.logits.softmax(-1).cpu()  # ロジットをソフトマックスで確率に変換
        
        a_win.extend(proba[:, 0].tolist())  # モデルAの勝率をリストに追加
        b_win.extend(proba[:, 1].tolist())  # モデルBの勝率をリストに追加
        tie.extend(proba[:, 2].tolist())  # 引き分けの確率をリストに追加
    
    df["winner_model_a"] = a_win  # モデルAの勝率をデータフレームに追加
    df["winner_model_b"] = b_win  # モデルBの勝率をデータフレームに追加
    df["winner_tie"] = tie  # 引き分けの確率をデータフレームに追加
    
    return df  # 更新されたデータフレームを返す
```

</div>
</details>

In [ ]:
@torch.no_grad()  # 勾配計算を無効にする
@torch.cuda.amp.autocast()  # 自動混合精度を使って演算を行う
def inference(df, model, device, batch_size=cfg.batch_size, max_length=cfg.max_length):
    a_win, b_win, tie = [], [], []  # 各モデルの勝率と引き分けを記録するリスト
    
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))  # バッチの終端インデックスを計算
        tmp = df.iloc[start_idx:end_idx]  # データフレームからバッチを取得
        input_ids = tmp["input_ids"].to_list()  # 入力IDを取得
        attention_mask = tmp["attention_mask"].to_list()  # アテンションマスクを取得
        # トークナイザーを使ってデータをパディング
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding="longest",
            pad_to_multiple_of=None,
            return_tensors="pt",
        )
        outputs = model(**inputs.to(device))  # モデルで出力を計算
        proba = outputs.logits.softmax(-1).cpu()  # ロジットをソフトマックスで確率に変換
        
        a_win.extend(proba[:, 0].tolist())  # モデルAの勝率をリストに追加
        b_win.extend(proba[:, 1].tolist())  # モデルBの勝率をリストに追加
        tie.extend(proba[:, 2].tolist())  # 引き分けの確率をリストに追加
    
    df["winner_model_a"] = a_win  # モデルAの勝率をデータフレームに追加
    df["winner_model_b"] = b_win  # モデルBの勝率をデータフレームに追加
    df["winner_tie"] = tie  # 引き分けの確率をデータフレームに追加
    
    return df  # 更新されたデータフレームを返す

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
st = time.time()

# sort by input length to fully leverage dynaminc padding
data = data.sort_values("length", ascending=False)
# the total #tokens in sub_1 and sub_2 should be more or less the same
sub_1 = data.iloc[0::2].copy()
sub_2 = data.iloc[1::2].copy()

with ThreadPoolExecutor(max_workers=2) as executor:
    results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))

result_df = pd.concat(list(results), axis=0)
proba = result_df[["winner_model_a", "winner_model_b", "winner_tie"]].values

print(f"elapsed time: {time.time() - st}")
```

</div>
<div class="column-right">

# 日本語訳

```python
st = time.time()  # 処理開始時間を記録

# 入力の長さでソートして動的パディングを最大限に活用する
data = data.sort_values("length", ascending=False)
# sub_1とsub_2のトークン数がほぼ同じになるように分ける
sub_1 = data.iloc[0::2].copy()  # 偶数番目のデータを選択
sub_2 = data.iloc[1::2].copy()  # 奇数番目のデータを選択

with ThreadPoolExecutor(max_workers=2) as executor:
    results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))  # 2つのモデルで推論実行

result_df = pd.concat(list(results), axis=0)  # 結果を結合
proba = result_df[["winner_model_a", "winner_model_b", "winner_tie"]].values  # 勝率の配列を取得

print(f"経過時間: {time.time() - st}")  # 処理時間を表示
```

</div>
</details>

In [ ]:
st = time.time()  # 処理開始時間を記録

# 入力の長さでソートして動的パディングを最大限に活用する
data = data.sort_values("length", ascending=False)
# sub_1とsub_2のトークン数がほぼ同じになるように分ける
sub_1 = data.iloc[0::2].copy()  # 偶数番目のデータを選択
sub_2 = data.iloc[1::2].copy()  # 奇数番目のデータを選択

with ThreadPoolExecutor(max_workers=2) as executor:
    results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))  # 2つのモデルで推論実行

result_df = pd.concat(list(results), axis=0)  # 結果を結合
proba = result_df[["winner_model_a", "winner_model_b", "winner_tie"]].values  # 勝率の配列を取得

print(f"経過時間: {time.time() - st}")  # 処理時間を表示

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
st = time.time()

if cfg.tta:
    data = aug_data.sort_values("length", ascending=False)  # sort by input length to boost speed
    sub_1 = data.iloc[0::2].copy()
    sub_2 = data.iloc[1::2].copy()

    with ThreadPoolExecutor(max_workers=2) as executor:
        results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))

    tta_result_df = pd.concat(list(results), axis=0)
    # recall TTA's order is flipped
    tta_proba = tta_result_df[["winner_model_b", "winner_model_a", "winner_tie"]].values 
    # average original result and TTA result.
    proba = (proba + tta_proba) / 2

print(f"elapsed time: {time.time() - st}")
```

</div>
<div class="column-right">

# 日本語訳

```python
st = time.time()

if cfg.tta:  # TTAが有効な場合
    data = aug_data.sort_values("length", ascending=False)  # 入力の長さでソートしてスピードを向上させる
    sub_1 = data.iloc[0::2].copy()  # 偶数番目のデータ
    sub_2 = data.iloc[1::2].copy()  # 奇数番目のデータ

    with ThreadPoolExecutor(max_workers=2) as executor:
        results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))  # 2つのモデルで推論実行

    tta_result_df = pd.concat(list(results), axis=0)  # TTAの結果を結合
    # TTAの順序が反転するので調整
    tta_proba = tta_result_df[["winner_model_b", "winner_model_a", "winner_tie"]].values 
    # 元の結果とTTA結果を平均
    proba = (proba + tta_proba) / 2

print(f"経過時間: {time.time() - st}")  # 処理時間を表示
```

</div>
</details>

In [ ]:
st = time.time()

if cfg.tta:  # TTAが有効な場合
    data = aug_data.sort_values("length", ascending=False)  # 入力の長さでソートしてスピードを向上させる
    sub_1 = data.iloc[0::2].copy()  # 偶数番目のデータ
    sub_2 = data.iloc[1::2].copy()  # 奇数番目のデータ

    with ThreadPoolExecutor(max_workers=2) as executor:
        results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))  # 2つのモデルで推論実行

    tta_result_df = pd.concat(list(results), axis=0)  # TTAの結果を結合
    # TTAの順序が反転するので調整
    tta_proba = tta_result_df[["winner_model_b", "winner_model_a", "winner_tie"]].values 
    # 元の結果とTTA結果を平均
    proba = (proba + tta_proba) / 2

print(f"経過時間: {time.time() - st}")  # 処理時間を表示

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
result_df.loc[:, "winner_model_a"] = proba[:, 0]
result_df.loc[:, "winner_model_b"] = proba[:, 1]
result_df.loc[:, "winner_tie"] = proba[:, 2]
submission_df = result_df[["id", 'winner_model_a', 'winner_model_b', 'winner_tie']]
submission_df.to_csv('submission.csv', index=False)
display(submission_df)
```

</div>
<div class="column-right">

# 日本語訳

```python
result_df.loc[:, "winner_model_a"] = proba[:, 0]  # モデルAの勝率を更新
result_df.loc[:, "winner_model_b"] = proba[:, 1]  # モデルBの勝率を更新
result_df.loc[:, "winner_tie"] = proba[:, 2]  # 引き分けの勝率を更新
submission_df = result_df[["id", 'winner_model_a', 'winner_model_b', 'winner_tie']]  # 提出用のデータフレームを作成
submission_df.to_csv('submission.csv', index=False)  # CSVファイルに保存
display(submission_df)  # 提出データを表示
```

</div>
</details>

In [ ]:
result_df.loc[:, "winner_model_a"] = proba[:, 0]  # モデルAの勝率を更新
result_df.loc[:, "winner_model_b"] = proba[:, 1]  # モデルBの勝率を更新
result_df.loc[:, "winner_tie"] = proba[:, 2]  # 引き分けの勝率を更新
submission_df = result_df[["id", 'winner_model_a', 'winner_model_b', 'winner_tie']]  # 提出用のデータフレームを作成
submission_df.to_csv('submission.csv', index=False)  # CSVファイルに保存
display(submission_df)  # 提出データを表示